<a href="https://colab.research.google.com/github/Suraj209211/PMOMEX---PREDICTIVE-MODEL-OF-MINERAL-EXPLORATION/blob/main/combinedcsv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# List of file paths
file_paths = [
    '/content/drive/MyDrive/PREDICTIVE MODULE OF MINERAL EXPLORATION/PMOMEX---PREDICTIVE-MODEL-OF-MINERAL-EXPLORATION/CODE/DATA-PREPROCESSING/csv/summary_statistics_0.csv',
    '/content/drive/MyDrive/PREDICTIVE MODULE OF MINERAL EXPLORATION/PMOMEX---PREDICTIVE-MODEL-OF-MINERAL-EXPLORATION/CODE/DATA-PREPROCESSING/csv/summary_statistics_1.csv',
    '/content/drive/MyDrive/PREDICTIVE MODULE OF MINERAL EXPLORATION/PMOMEX---PREDICTIVE-MODEL-OF-MINERAL-EXPLORATION/CODE/DATA-PREPROCESSING/csv/summary_statistics_2.csv',
    '/content/drive/MyDrive/PREDICTIVE MODULE OF MINERAL EXPLORATION/PMOMEX---PREDICTIVE-MODEL-OF-MINERAL-EXPLORATION/CODE/DATA-PREPROCESSING/csv/summary_statistics_3.csv',
    '/content/drive/MyDrive/PREDICTIVE MODULE OF MINERAL EXPLORATION/PMOMEX---PREDICTIVE-MODEL-OF-MINERAL-EXPLORATION/CODE/DATA-PREPROCESSING/csv/summary_statistics_4.csv',
    '/content/drive/MyDrive/PREDICTIVE MODULE OF MINERAL EXPLORATION/PMOMEX---PREDICTIVE-MODEL-OF-MINERAL-EXPLORATION/CODE/DATA-PREPROCESSING/csv/summary_statistics_5.csv',
    '/content/drive/MyDrive/PREDICTIVE MODULE OF MINERAL EXPLORATION/PMOMEX---PREDICTIVE-MODEL-OF-MINERAL-EXPLORATION/CODE/DATA-PREPROCESSING/csv/summary_statistics_6.csv',
    '/content/drive/MyDrive/PREDICTIVE MODULE OF MINERAL EXPLORATION/PMOMEX---PREDICTIVE-MODEL-OF-MINERAL-EXPLORATION/CODE/DATA-PREPROCESSING/csv/summary_statistics_7.csv',
    '/content/drive/MyDrive/PREDICTIVE MODULE OF MINERAL EXPLORATION/PMOMEX---PREDICTIVE-MODEL-OF-MINERAL-EXPLORATION/CODE/DATA-PREPROCESSING/csv/summary_statistics_8.csv',
    '/content/drive/MyDrive/PREDICTIVE MODULE OF MINERAL EXPLORATION/PMOMEX---PREDICTIVE-MODEL-OF-MINERAL-EXPLORATION/CODE/DATA-PREPROCESSING/csv/summary_statistics_9.csv',
    '/content/drive/MyDrive/PREDICTIVE MODULE OF MINERAL EXPLORATION/PMOMEX---PREDICTIVE-MODEL-OF-MINERAL-EXPLORATION/CODE/DATA-PREPROCESSING/csv/summary_statistics_10.csv',
    '/content/drive/MyDrive/PREDICTIVE MODULE OF MINERAL EXPLORATION/PMOMEX---PREDICTIVE-MODEL-OF-MINERAL-EXPLORATION/CODE/DATA-PREPROCESSING/csv/summary_statistics_11.csv',
    '/content/drive/MyDrive/PREDICTIVE MODULE OF MINERAL EXPLORATION/PMOMEX---PREDICTIVE-MODEL-OF-MINERAL-EXPLORATION/CODE/DATA-PREPROCESSING/csv/summary_statistics_12.csv',
]

# Load and concatenate all dataframes
dfs = [pd.read_csv(file_path) for file_path in file_paths]
combined_df = pd.concat(dfs, ignore_index=True)

# Calculate aggregate statistics for the combined data
aggregate_stats = combined_df.describe()

# Save the combined data to a new CSV file
combined_file_path = '/content/drive/MyDrive/PREDICTIVE MODULE OF MINERAL EXPLORATION/PMOMEX---PREDICTIVE-MODEL-OF-MINERAL-EXPLORATION/CODE/DATA-PREPROCESSING/csv/combined_summary_statistics.csv'
combined_df.to_csv(combined_file_path, index=False)

# Save the aggregate statistics to a new CSV file
aggregate_stats_file_path = '/content/drive/MyDrive/PREDICTIVE MODULE OF MINERAL EXPLORATION/PMOMEX---PREDICTIVE-MODEL-OF-MINERAL-EXPLORATION/CODE/DATA-PREPROCESSING/csv/aggregate_summary_statistics.csv'
aggregate_stats.to_csv(aggregate_stats_file_path)

# Print the file paths
print("File Paths:")
for file_path in file_paths:
    print(file_path)


In [ ]:
print("Aggregate Statistics:\n", aggregate_stats)

# Output the file paths for verification
print("Combined file saved at:", combined_file_path)
print("Aggregate statistics file saved at:", aggregate_stats_file_path)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the combined data
combined_file_path = '/content/drive/MyDrive/PREDICTIVE MODULE OF MINERAL EXPLORATION/PMOMEX---PREDICTIVE-MODEL-OF-MINERAL-EXPLORATION/CODE/DATA-PREPROCESSING/csv/aggregate_summary_statistics.csv'
combined_df = pd.read_csv(combined_file_path)

# Drop non-numeric columns and metadata rows if any
numeric_df = combined_df.select_dtypes(include=['number'])

# Calculate the correlation matrix
correlation_matrix = numeric_df.corr()

# Generate a heatmap
plt.figure(figsize=(50,50))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap of Combined Data')
plt.show()



In [ ]:
import os
import geopandas as gpd
import pandas as pd

# Specify the directory where your CSV files are located
folder_path = '/content/drive/MyDrive/PREDICTIVE MODULE OF MINERAL EXPLORATION/PMOMEX---PREDICTIVE-MODEL-OF-MINERAL-EXPLORATION/NEW_DATA/Geochemical/geocsv/SOIL SEDIMENT SAMPLE DATA'  # Replace with your folder path

# Initialize an empty list to hold GeoDataFrames
gdfs = []

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Check if 'Longitude' and 'Latitude' columns exist, handle cases where they might be named differently or absent
        longitude_column = 'Longitude' if 'Longitude' in df.columns else (next((col for col in df.columns if 'long' in col.lower()), None))
        latitude_column = 'Latitude' if 'Latitude' in df.columns else (next((col for col in df.columns if 'lat' in col.lower()), None))

        if longitude_column and latitude_column:
            # Attempt to convert longitude and latitude columns to numeric, coercing errors to NaN
            df[longitude_column] = pd.to_numeric(df[longitude_column], errors='coerce')
            df[latitude_column] = pd.to_numeric(df[latitude_column], errors='coerce')

            # Drop rows with NaN values in longitude or latitude
            df.dropna(subset=[longitude_column, latitude_column], inplace=True)

            # Convert the DataFrame to a GeoDataFrame
            gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[longitude_column], df[latitude_column]))
            gdf.set_crs(epsg=4326, inplace=True)  # Set the CRS to WGS84 (EPSG:4326)

            # Append the GeoDataFrame to the list
            gdfs.append(gdf)
        else:
            print(f"Warning: Skipping file {filename} due to missing or differently named Longitude/Latitude columns.")

# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
if gdfs:
    combined_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))

    # Specify the output file path, including the file name
    output_file = '/content/drive/MyDrive/PREDICTIVE MODULE OF MINERAL EXPLORATION/PMOMEX---PREDICTIVE-MODEL-OF-MINERAL-EXPLORATION/NEW_DATA/Geochemical/geocsv/combined_soil_sediment_sample_data.csv'

    # Save the combined GeoDataFrame to a new CSV file
    combined_gdf.to_csv(output_file, index=False)

    print(f"Combined CSV saved to {output_file}")
else:
    print("No GeoDataFrames were created. Check the input files and processing logic.")


In [ ]:
import os
import geopandas as gpd
import pandas as pd

# Specify the directory where your CSV files are located
folder_path = '/content/drive/MyDrive/PREDICTIVE MODULE OF MINERAL EXPLORATION/PMOMEX---PREDICTIVE-MODEL-OF-MINERAL-EXPLORATION/NEW_DATA/Geochemical/geocsv/SOIL SEDIMENT SAMPLE DATA'  # Replace with your folder path

# Initialize an empty list to hold GeoDataFrames
gdfs = []

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Check if 'Longitude' and 'Latitude' columns exist, handle cases where they might be named differently or absent
        longitude_column = 'Longitude' if 'Longitude' in df.columns else (next((col for col in df.columns if 'long' in col.lower()), None))
        latitude_column = 'Latitude' if 'Latitude' in df.columns else (next((col for col in df.columns if 'lat' in col.lower()), None))

        if longitude_column and latitude_column:
            # Attempt to convert longitude and latitude columns to numeric, coercing errors to NaN
            df[longitude_column] = pd.to_numeric(df[longitude_column], errors='coerce')
            df[latitude_column] = pd.to_numeric(df[latitude_column], errors='coerce')

            # Drop rows with NaN values in longitude or latitude
            df.dropna(subset=[longitude_column, latitude_column], inplace=True)

            # Convert the DataFrame to a GeoDataFrame
            gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[longitude_column], df[latitude_column]))
            gdf.set_crs(epsg=4326, inplace=True)  # Set the CRS to WGS84 (EPSG:4326)

            # Append the GeoDataFrame to the list
            gdfs.append(gdf)
        else:
            print(f"Warning: Skipping file {filename} due to missing or differently named Longitude/Latitude columns.")

# Handle files with different column names individually (if needed)
# Example for 'Copy of 45L02_Water.csv':
water_file_path = os.path.join(folder_path, 'Copy of 45L02_Water.csv')
if os.path.exists(water_file_path):
    df_water = pd.read_csv(water_file_path)
    if 'Long' in df_water.columns and 'Lat' in df_water.columns:
        df_water['Long'] = pd.to_numeric(df_water['Long'], errors='coerce')
        df_water['Lat'] = pd.to_numeric(df_water['Lat'], errors='coerce')
        df_water.dropna(subset=['Long', 'Lat'], inplace=True)
        gdf_water = gpd.GeoDataFrame(df_water, geometry=gpd.points_from_xy(df_water['Long'], df_water['Lat']))
        gdf_water.set_crs(epsg=4326, inplace=True)
        gdfs.append(gdf_water)
    else:
        print("Warning: 'Copy of 45L02_Water.csv' does not have expected 'Long' and 'Lat' columns.")

# Repeat the above process for other skipped files with known column names

# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
if gdfs:
    combined_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))

    # Specify the output file path, including the file name
    output_file = '/content/drive/MyDrive/PREDICTIVE MODULE OF MINERAL EXPLORATION/PMOMEX---PREDICTIVE-MODEL-OF-MINERAL-EXPLORATION/NEW_DATA/Geochemical/geocsv/combined_soil_sediment_sample_data.csv'

    # Save the combined GeoDataFrame to a new CSV file
    combined_gdf.to_csv(output_file, index=False)

    print(f"Combined CSV saved to {output_file}")
else:
    print("No GeoDataFrames were created. Check the input files and processing logic.")

In [ ]:
import os
import geopandas as gpd
import pandas as pd

# Specify the directory where your CSV files are located
folder_path = '/content/drive/MyDrive/PREDICTIVE MODULE OF MINERAL EXPLORATION/PMOMEX---PREDICTIVE-MODEL-OF-MINERAL-EXPLORATION/NEW_DATA/Geochemical/geocsv/SOIL SAMPLE'  # Replace with your folder path

# Initialize an empty list to hold GeoDataFrames
gdfs = []

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Dynamically find longitude and latitude columns (case-insensitive)
        longitude_column = next((col for col in df.columns if 'long' in col.lower()), None)
        latitude_column = next((col for col in df.columns if 'lat' in col.lower()), None)

        if longitude_column and latitude_column:
            # Attempt to convert longitude and latitude columns to numeric, coercing errors to NaN
            df[longitude_column] = pd.to_numeric(df[longitude_column], errors='coerce')
            df[latitude_column] = pd.to_numeric(df[latitude_column], errors='coerce')

            # Drop rows with NaN values in longitude or latitude
            df.dropna(subset=[longitude_column, latitude_column], inplace=True)

            # Convert the DataFrame to a GeoDataFrame using the dynamically found columns
            gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[longitude_column], df[latitude_column]))
            gdf.set_crs(epsg=4326, inplace=True)  # Set the CRS to WGS84 (EPSG:4326)

            # Append the GeoDataFrame to the list
            gdfs.append(gdf)
        else:
            print(f"Warning: Skipping file {filename} due to missing or unrecognized Longitude/Latitude columns.")

# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
if gdfs:
    combined_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))

    # Specify the output file path, including the file name
    output_file = '/content/drive/MyDrive/PREDICTIVE MODULE OF MINERAL EXPLORATION/PMOMEX---PREDICTIVE-MODEL-OF-MINERAL-EXPLORATION/NEW_DATA/Geochemical/geocsv/SOIL SAMPLE/combined_soil_sample_data.csv' # Modified to save in the SOIL SAMPLE folder

    # Save the combined GeoDataFrame to a new CSV file
    combined_gdf.to_csv(output_file, index=False)

    print(f"Combined CSV saved to {output_file}")
else:
    print("No GeoDataFrames were created. Check the input files and processing logic.")